Script to compute yearly mean annual precipitation on SAFRAN moving sub-domains

In [9]:
import sys
import os
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from config import *
from functions import get_data_yr_SAFR, load_cum_year
from utils import *

In [20]:
### CST ###

reg = 'Fr'
dataset = 'SAFRAN'
ymin = 1958
ymax = 2021
years = np.arange(ymin, ymax+1, 1)
per = str(ymin) + '-' + str(ymax)
size = 60  # domain side length (number of pixels)
size_ = str(int(size*res/1000)) + 'x' + str(int(size*res/1000))
prop = 0.33  # proportion du côté de la fenêtre sans recouvrement (dans les deux directions)
step = int(round(size * prop, 0))  # décalage dans la sélection des coordonnées des sous-fenêtres
ntot = size **2  # total number of pixel within domain
th_n = 0.75  # minimum proportion of land grid cell for the sub-domain to be considered

In [21]:
#~ OUTDIR

if not os.path.isdir(DATADIR + '/' + reg):
    os.mkdir(DATADIR + '/' + reg)
datadir = DATADIR + '/' + reg

if not os.path.isdir(datadir + '/' + dataset):
    os.mkdir(datadir + '/' + dataset)
datadir = datadir + '/' + dataset

if not os.path.isdir(datadir + '/' + per):
    os.mkdir(datadir + '/' + per)
datadir = datadir + '/' + per

if not os.path.isdir(datadir + '/cum'):
    os.mkdir(datadir + '/cum')
datadir = datadir + '/cum'

if not os.path.isdir(datadir + '/' + size_):
    os.mkdir(datadir + '/' + size_)
datadir = datadir + '/' + size_

print(datadir)

/home/guillaumechagnaud/TRAVAIL/POSTDOC/IGE-Secheresse/work/output/data/Fr/SAFRAN/1958-2021/cum/480x480


In [22]:
# Min and max coordinates (x, y) of the lower left pixel of sub-domains
x_min_ = x_min
x_max_ = x_max - size * res - res
y_min_ = y_min
y_max_ = y_max - size * res - res
(x_min_, y_min_), (x_max_, y_max_)

((60000, 1617000), (708000, 2193000))

In [23]:
# Coordinates of the lower left pixel of all sub-domains
xs = pd.Series(np.arange(x_min_, x_max_+res, res))  # all x coordinates
ys = pd.Series(np.arange(y_min_, y_max_+res, res))  # all y coordinates

In [6]:
# Sub-sample sub-domains coordinates (-> over-lapping)
xs_ = xs #[::step]
ys_ = ys #[::step]

In [24]:
x_centers = xs + size * res / 2 - res / 2
y_centers = ys + size * res / 2 - res / 2

In [26]:
# Get all sub-domains lower left (ll) coordinates (x, y)
ll_cells = []
for x in xs:
    for y in ys:
        ll_cells.append((x, y))
len(ll_cells)

5986

In [8]:
# Get valid cells

ds0 = get_data_yr_SAFR(1958)
valid_cells = []

for i, ll_cell in enumerate(ll_cells):
    x_min_ = ll_cell[0]
    x_max_ = x_min_ + size * res - res
    y_min_ = ll_cell[1]
    y_max_ = y_min_ + size * res - res
    xs = [x_min_, x_max_]
    ys = [y_min_, y_max_]
    data = ds0.sel(x=slice(xs[0], xs[1]), y=slice(ys[0], ys[1]))

    if len(data.product.values.mean(axis=0)[~np.isnan(data.product.values.mean(axis=0))]) >= ntot*th_n:
        print('{0}/{1}'.format((i+1), len(ll_cells)), end=' : ', flush=True)
        #print(xs, end=' : ', flush=True)
        valid_cells.append(ll_cell)

885/7636 : 886/7636 : 887/7636 : 888/7636 : 889/7636 : 890/7636 : 891/7636 : 892/7636 : 965/7636 : 966/7636 : 967/7636 : 968/7636 : 969/7636 : 970/7636 : 971/7636 : 972/7636 : 973/7636 : 974/7636 : 975/7636 : 976/7636 : 977/7636 : 978/7636 : 979/7636 : 1046/7636 : 1047/7636 : 1048/7636 : 1049/7636 : 1050/7636 : 1051/7636 : 1052/7636 : 1053/7636 : 1054/7636 : 1055/7636 : 1056/7636 : 1057/7636 : 1058/7636 : 1059/7636 : 1060/7636 : 1061/7636 : 1062/7636 : 1063/7636 : 1127/7636 : 1128/7636 : 1129/7636 : 1130/7636 : 1131/7636 : 1132/7636 : 1133/7636 : 1134/7636 : 1135/7636 : 1136/7636 : 1137/7636 : 1138/7636 : 1139/7636 : 1140/7636 : 1141/7636 : 1142/7636 : 1143/7636 : 1144/7636 : 1145/7636 : 1146/7636 : 1147/7636 : 1148/7636 : 1208/7636 : 1209/7636 : 1210/7636 : 1211/7636 : 1212/7636 : 1213/7636 : 1214/7636 : 1215/7636 : 1216/7636 : 1217/7636 : 1218/7636 : 1219/7636 : 1220/7636 : 1221/7636 : 1222/7636 : 1223/7636 : 1224/7636 : 1225/7636 : 1226/7636 : 1227/7636 : 1228/7636 : 1229/7636 : 123

2172/7636 : 2173/7636 : 2174/7636 : 2175/7636 : 2176/7636 : 2177/7636 : 2178/7636 : 2179/7636 : 2180/7636 : 2181/7636 : 2182/7636 : 2183/7636 : 2184/7636 : 2185/7636 : 2186/7636 : 2187/7636 : 2188/7636 : 2189/7636 : 2190/7636 : 2191/7636 : 2192/7636 : 2193/7636 : 2194/7636 : 2195/7636 : 2196/7636 : 2197/7636 : 2198/7636 : 2199/7636 : 2200/7636 : 2201/7636 : 2202/7636 : 2203/7636 : 2204/7636 : 2205/7636 : 2206/7636 : 2207/7636 : 2208/7636 : 2209/7636 : 2210/7636 : 2211/7636 : 2212/7636 : 2213/7636 : 2214/7636 : 2215/7636 : 2216/7636 : 2217/7636 : 2218/7636 : 2219/7636 : 2220/7636 : 2221/7636 : 2222/7636 : 2223/7636 : 2224/7636 : 2225/7636 : 2226/7636 : 2227/7636 : 2228/7636 : 2229/7636 : 2230/7636 : 2231/7636 : 2232/7636 : 2233/7636 : 2234/7636 : 2235/7636 : 2236/7636 : 2237/7636 : 2238/7636 : 2249/7636 : 2250/7636 : 2251/7636 : 2252/7636 : 2253/7636 : 2254/7636 : 2255/7636 : 2256/7636 : 2257/7636 : 2258/7636 : 2259/7636 : 2260/7636 : 2261/7636 : 2262/7636 : 2263/7636 : 2264/7636 : 2265

2902/7636 : 2903/7636 : 2904/7636 : 2905/7636 : 2911/7636 : 2912/7636 : 2913/7636 : 2914/7636 : 2915/7636 : 2916/7636 : 2917/7636 : 2918/7636 : 2919/7636 : 2920/7636 : 2921/7636 : 2922/7636 : 2923/7636 : 2924/7636 : 2925/7636 : 2926/7636 : 2927/7636 : 2928/7636 : 2929/7636 : 2930/7636 : 2931/7636 : 2932/7636 : 2933/7636 : 2934/7636 : 2935/7636 : 2936/7636 : 2937/7636 : 2938/7636 : 2939/7636 : 2940/7636 : 2941/7636 : 2942/7636 : 2943/7636 : 2944/7636 : 2945/7636 : 2946/7636 : 2947/7636 : 2948/7636 : 2949/7636 : 2950/7636 : 2951/7636 : 2952/7636 : 2953/7636 : 2954/7636 : 2955/7636 : 2956/7636 : 2957/7636 : 2958/7636 : 2959/7636 : 2960/7636 : 2961/7636 : 2962/7636 : 2963/7636 : 2964/7636 : 2965/7636 : 2966/7636 : 2967/7636 : 2968/7636 : 2969/7636 : 2970/7636 : 2971/7636 : 2972/7636 : 2973/7636 : 2974/7636 : 2975/7636 : 2976/7636 : 2977/7636 : 2978/7636 : 2979/7636 : 2980/7636 : 2981/7636 : 2982/7636 : 2983/7636 : 2984/7636 : 2985/7636 : 2986/7636 : 2987/7636 : 2988/7636 : 2994/7636 : 2995

3633/7636 : 3634/7636 : 3635/7636 : 3636/7636 : 3637/7636 : 3638/7636 : 3639/7636 : 3640/7636 : 3641/7636 : 3642/7636 : 3643/7636 : 3644/7636 : 3645/7636 : 3646/7636 : 3647/7636 : 3648/7636 : 3649/7636 : 3650/7636 : 3651/7636 : 3652/7636 : 3659/7636 : 3660/7636 : 3661/7636 : 3662/7636 : 3663/7636 : 3664/7636 : 3665/7636 : 3666/7636 : 3667/7636 : 3668/7636 : 3669/7636 : 3670/7636 : 3671/7636 : 3672/7636 : 3673/7636 : 3674/7636 : 3675/7636 : 3676/7636 : 3677/7636 : 3678/7636 : 3679/7636 : 3680/7636 : 3681/7636 : 3682/7636 : 3683/7636 : 3684/7636 : 3685/7636 : 3686/7636 : 3687/7636 : 3688/7636 : 3689/7636 : 3690/7636 : 3691/7636 : 3692/7636 : 3693/7636 : 3694/7636 : 3695/7636 : 3696/7636 : 3697/7636 : 3698/7636 : 3699/7636 : 3700/7636 : 3701/7636 : 3702/7636 : 3703/7636 : 3704/7636 : 3705/7636 : 3706/7636 : 3707/7636 : 3708/7636 : 3709/7636 : 3710/7636 : 3711/7636 : 3712/7636 : 3713/7636 : 3714/7636 : 3715/7636 : 3716/7636 : 3717/7636 : 3718/7636 : 3719/7636 : 3720/7636 : 3721/7636 : 3722

4376/7636 : 4377/7636 : 4378/7636 : 4379/7636 : 4380/7636 : 4381/7636 : 4382/7636 : 4383/7636 : 4384/7636 : 4385/7636 : 4386/7636 : 4387/7636 : 4388/7636 : 4389/7636 : 4390/7636 : 4391/7636 : 4392/7636 : 4393/7636 : 4394/7636 : 4395/7636 : 4396/7636 : 4397/7636 : 4398/7636 : 4399/7636 : 4407/7636 : 4408/7636 : 4409/7636 : 4410/7636 : 4411/7636 : 4412/7636 : 4413/7636 : 4414/7636 : 4415/7636 : 4416/7636 : 4417/7636 : 4418/7636 : 4419/7636 : 4420/7636 : 4421/7636 : 4422/7636 : 4423/7636 : 4424/7636 : 4425/7636 : 4426/7636 : 4427/7636 : 4428/7636 : 4429/7636 : 4430/7636 : 4431/7636 : 4432/7636 : 4433/7636 : 4434/7636 : 4435/7636 : 4436/7636 : 4437/7636 : 4438/7636 : 4439/7636 : 4440/7636 : 4441/7636 : 4442/7636 : 4443/7636 : 4444/7636 : 4445/7636 : 4446/7636 : 4447/7636 : 4448/7636 : 4449/7636 : 4450/7636 : 4451/7636 : 4452/7636 : 4453/7636 : 4454/7636 : 4455/7636 : 4456/7636 : 4457/7636 : 4458/7636 : 4459/7636 : 4460/7636 : 4461/7636 : 4462/7636 : 4463/7636 : 4464/7636 : 4465/7636 : 4466

5131/7636 : 5132/7636 : 5133/7636 : 5134/7636 : 5135/7636 : 5136/7636 : 5137/7636 : 5138/7636 : 5139/7636 : 5140/7636 : 5141/7636 : 5142/7636 : 5143/7636 : 5144/7636 : 5145/7636 : 5146/7636 : 5156/7636 : 5157/7636 : 5158/7636 : 5159/7636 : 5160/7636 : 5161/7636 : 5162/7636 : 5163/7636 : 5164/7636 : 5165/7636 : 5166/7636 : 5167/7636 : 5168/7636 : 5169/7636 : 5170/7636 : 5171/7636 : 5172/7636 : 5173/7636 : 5174/7636 : 5175/7636 : 5176/7636 : 5177/7636 : 5178/7636 : 5179/7636 : 5180/7636 : 5181/7636 : 5182/7636 : 5183/7636 : 5184/7636 : 5185/7636 : 5186/7636 : 5187/7636 : 5188/7636 : 5189/7636 : 5190/7636 : 5191/7636 : 5192/7636 : 5193/7636 : 5194/7636 : 5195/7636 : 5196/7636 : 5197/7636 : 5198/7636 : 5199/7636 : 5200/7636 : 5201/7636 : 5202/7636 : 5203/7636 : 5204/7636 : 5205/7636 : 5206/7636 : 5207/7636 : 5208/7636 : 5209/7636 : 5210/7636 : 5211/7636 : 5212/7636 : 5213/7636 : 5214/7636 : 5215/7636 : 5216/7636 : 5217/7636 : 5218/7636 : 5219/7636 : 5220/7636 : 5221/7636 : 5222/7636 : 5223

5949/7636 : 5950/7636 : 5951/7636 : 5952/7636 : 5953/7636 : 5954/7636 : 5955/7636 : 5956/7636 : 5957/7636 : 5958/7636 : 5959/7636 : 5960/7636 : 5961/7636 : 5962/7636 : 5963/7636 : 5964/7636 : 5965/7636 : 5966/7636 : 5967/7636 : 5968/7636 : 5969/7636 : 5994/7636 : 5995/7636 : 5996/7636 : 5997/7636 : 5998/7636 : 5999/7636 : 6000/7636 : 6001/7636 : 6002/7636 : 6003/7636 : 6004/7636 : 6005/7636 : 6006/7636 : 6007/7636 : 6008/7636 : 6009/7636 : 6010/7636 : 6011/7636 : 6012/7636 : 6013/7636 : 6014/7636 : 6015/7636 : 6016/7636 : 6017/7636 : 6018/7636 : 6019/7636 : 6020/7636 : 6021/7636 : 6022/7636 : 6023/7636 : 6024/7636 : 6025/7636 : 6026/7636 : 6027/7636 : 6028/7636 : 6029/7636 : 6030/7636 : 6031/7636 : 6032/7636 : 6033/7636 : 6034/7636 : 6035/7636 : 6036/7636 : 6037/7636 : 6038/7636 : 6039/7636 : 6040/7636 : 6041/7636 : 6042/7636 : 6043/7636 : 6044/7636 : 6045/7636 : 6046/7636 : 6047/7636 : 6048/7636 : 6049/7636 : 6050/7636 : 6079/7636 : 6080/7636 : 6081/7636 : 6082/7636 : 6083/7636 : 6084

In [9]:
#~ Compute yearly accumulation

print('-- Compute accumulation --')

for y in years:
    print('\n>>> %i <<<'% y)

    ds = get_data_yr_SAFR(y)

    cums = {}

    for i, ll_cell in enumerate(valid_cells):
        #print('{0}/{1}'.format((i+1), len(ll_cells)), end=' : ', flush=True)

        x_min_ = ll_cell[0]
        x_max_ = x_min_ + size * res - res
        y_min_ = ll_cell[1]
        y_max_ = y_min_ + size * res - res

        data = ds.sel(x=slice(x_min_, x_max_), y=slice(y_min_, y_max_))
        cum = data.mean(dim='time').product * 3600 * len(data.indexes['time'])  # hourly mean flux -> mm/h -> mm/y
        cums[(x_min_, y_min_)] = cum.mean()  # spatial mean

    datafile = datadir + '/' + str(y)

    with open(datafile, 'wb') as pics:
        pickle.dump(file=pics, obj=cums)


print('Done')


>>> 1958 <<<

>>> 1959 <<<

>>> 1960 <<<

>>> 1961 <<<

>>> 1962 <<<

>>> 1963 <<<

>>> 1964 <<<

>>> 1965 <<<

>>> 1966 <<<

>>> 1967 <<<

>>> 1968 <<<

>>> 1969 <<<

>>> 1970 <<<

>>> 1971 <<<

>>> 1972 <<<

>>> 1973 <<<

>>> 1974 <<<

>>> 1975 <<<

>>> 1976 <<<

>>> 1977 <<<

>>> 1978 <<<

>>> 1979 <<<

>>> 1980 <<<

>>> 1981 <<<

>>> 1982 <<<

>>> 1983 <<<

>>> 1984 <<<

>>> 1985 <<<

>>> 1986 <<<

>>> 1987 <<<

>>> 1988 <<<

>>> 1989 <<<

>>> 1990 <<<

>>> 1991 <<<

>>> 1992 <<<

>>> 1993 <<<

>>> 1994 <<<

>>> 1995 <<<

>>> 1996 <<<

>>> 1997 <<<

>>> 1998 <<<

>>> 1999 <<<

>>> 2000 <<<

>>> 2001 <<<

>>> 2002 <<<

>>> 2003 <<<

>>> 2004 <<<

>>> 2005 <<<

>>> 2006 <<<

>>> 2007 <<<

>>> 2008 <<<

>>> 2009 <<<

>>> 2010 <<<

>>> 2011 <<<

>>> 2012 <<<

>>> 2013 <<<

>>> 2014 <<<

>>> 2015 <<<

>>> 2016 <<<

>>> 2017 <<<

>>> 2018 <<<

>>> 2019 <<<

>>> 2020 <<<

>>> 2021 <<<
Done


In [27]:
#~ Merge years

out_years = []
    
for yr in years:
    print(yr, end=' : ', flush=True)

    cum = load_cum_year(ymin, ymax, yr, size)
        
    valid_cells = list(cum.keys())

    out_ys = []
    for y in ys:
        out_xs = []
        for x in xs:
            ll_cell = (x, y)
            if ll_cell in valid_cells:
                out_xs.append(cum[ll_cell])
            else:
                out_xs.append(np.nan)
        out_ys.append(out_xs)

    out_ys = np.asarray(out_ys)
    out_ = xr.DataArray(data=out_ys, dims=['y', 'x'], coords=dict(x=(['x'], x_centers.values), y=(['y'], y_centers.values)))
    out_years.append(out_)
        
out = xr.concat(out_years, dim='year')
out = out.assign_coords(year=('year', years))


#~ Save

datafile = datadir + '/climato.nc'

out.to_netcdf(datafile)

print('Done')

1958 : 1959 : 1960 : 1961 : 1962 : 1963 : 1964 : 1965 : 1966 : 1967 : 1968 : 1969 : 1970 : 1971 : 1972 : 1973 : 1974 : 1975 : 1976 : 1977 : 1978 : 1979 : 1980 : 1981 : 1982 : 1983 : 1984 : 1985 : 1986 : 1987 : 1988 : 1989 : 1990 : 1991 : 1992 : 1993 : 1994 : 1995 : 1996 : 1997 : 1998 : 1999 : 2000 : 2001 : 2002 : 2003 : 2004 : 2005 : 2006 : 2007 : 2008 : 2009 : 2010 : 2011 : 2012 : 2013 : 2014 : 2015 : 2016 : 2017 : 2018 : 2019 : 2020 : 2021 : 